In [33]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize

In [175]:
potential = 'AHO'
cutoff = 16
shots = 1024

In [176]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [177]:
min_eigenvalue

np.float64(-0.001166975680574785)

In [178]:
cnot_pool = []
cz_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [179]:
rot_pool = [qml.Rot(0.0, 0.0, 0.0, wires=x) for x in range(num_qubits)]
operator_pool = rot_pool + cnot_pool + cz_pool

In [180]:
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    if len(op_list) > 0:
        for o, p, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(*p, wires=w)

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(*params, wires=trial_op.wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [181]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [183]:
num_steps = 50

x0 = np.random.uniform(0, 2 * np.pi, size=3)

op_list = []

for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []

    for trial_op in operator_pool:

        res = minimize(
                cost_function,
                x0,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':10000, 'tol': 1e-8}
            )
        
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg]

    if (i != 0):
        if np.abs(min_energy - op_list[i-1][3]) < 1e-6:
            print("Converged")
            break

    op_list.append((min_op, min_params, min_wires, min_energy))

    

step: 0
Min energy: 0.9374999999999998
step: 1
Min energy: 0.12682403352835273
step: 2
Min energy: 0.09428039469030801
step: 3
Min energy: 0.075689725284025
step: 4
Min energy: 0.05295062892024526
step: 5
Min energy: 0.050250669474299736
step: 6
Min energy: 0.04782075506601122
step: 7
Min energy: 0.042076003134798146
step: 8
Min energy: 0.037909467533078806
step: 9
Min energy: 0.03532674721295419
step: 10
Min energy: 0.03342641140241899
step: 11
Min energy: 0.03223126054912564
step: 12
Min energy: 0.03130093748208953
step: 13
Min energy: 0.030019505414910296
step: 14
Min energy: 0.029704830405905557
step: 15
Min energy: 0.029192367410515445
step: 16
Min energy: 0.028781433155971703
step: 17
Min energy: 0.02845411989782407
step: 18
Min energy: 0.02829606173061112
step: 19
Min energy: 0.028177567846937677
step: 20
Min energy: 0.02794426308929828
step: 21
Min energy: 0.027714637379158996
step: 22
Min energy: 0.027582603425228974
step: 23
Min energy: 0.027506572046139696
step: 24
Min energ

KeyboardInterrupt: 

In [184]:
op_list

[(pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([6.00319662, 3.14159265, 4.40759165]),
  Wires([0]),
  np.float64(0.9374999999999998)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([5.06154705, 5.96410452, 6.28318532]),
  Wires([3]),
  np.float64(0.12682403352835273)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([4.99867907, 6.24663565, 3.14215088]),
  Wires([2]),
  np.float64(0.09428039469030801)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([5.64729459, 6.22685391, 6.83096845]),
  Wires([3]),
  np.float64(0.075689725284025)),
 (pennylane.ops.op_math.controlled_ops.CZ,
  array([5.01245512, 4.91953565, 4.39497736]),
  Wires([3, 2]),
  np.float64(0.05295062892024526)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([6.02049008, 6.27893093, 5.62384941]),
  Wires([1]),
  np.float64(0.050250669474299736)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.Rot,
  array([5.69052949, 6.26287505, 6.84648689]

In [73]:
print(qml.draw(circuit)(pnp.tensor(min_params, requires_grad=True), trial_op))

0: ──Rot(1.19,3.14,4.13)─┤ ╭<𝓗(M0)>
1: ──Rot(1.19,3.14,4.13)─┤ ╰<𝓗(M0)>

M0 = 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [31]:
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def circuit():

    for o in op_list:
        type(o)(*o.data, wires=o.wires)

    op = type(operator_pool[0])
    op(*operator_pool[0].data, wires=operator_pool[0].wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))